19. Lậplịchthiđấuthểthao:
Có N đội tuyển 1, 2, ..., N cần được lập lịch thi đấu với nhau theo thể thức vòng tròn: 
- Mỗi đội phải gặp tất cả N-1 đội còn lại, 
- Mỗi đội 2 trận(lượt đi và lượt về trên sân nhà và trên sân khách) trong khoảng thời gian 2N-2 tuần, tuần nào cũng phải thi đấu đúng 1 trận. 
- Khoảng cách từ sân của đội tuyển i đến sân của đội tuyển j là d(i, j).
Hãy lập kế hoạch thi đấu cho N đội, sao cho: 
                            tổng khoảng cách di chuyển của các đội là ngắn nhất.

•Inpu
•Dòng1: ghi số nguyên dương N
•Dòngi+1 (i= 1,..., N): ghi hàng thứ i của ma trận d   
< Khoảngcáchtừsâncủađộituyểniđếnsâncủađộituyểnj là d(i,j).>

2
0 1
1 0

3
0 1 2
1 0 3
2 3 0

4
0 1 2 3
1 0 4 5
2 4 0 6
3 5 6 0

Found better configuration, the new best value is : 48
The new best config is [[0, 6, 2, 1], [3, 0, 1, 2], [5, 4, 0, 3], [4, 5, 6, 0]]


In [377]:
### gen_distance_matrix ###
import random as rd

def gen_distance_matrix(n):
    distance_matrix = [[rd.randint(1, 9) for i in range(n)] for j in range(n)]
    for i in range(n):
        distance_matrix[i][i] = 0
    for i in range(n): 
        for j in range(n): 
            distance_matrix[i][j] = distance_matrix[j][i]
    return distance_matrix    # print matrix

n = 6
distance_matrix = gen_distance_matrix(n)
print(distance_matrix)

[[0, 5, 9, 9, 4, 1], [5, 0, 8, 4, 1, 2], [9, 8, 0, 7, 3, 7], [9, 4, 7, 0, 6, 9], [4, 1, 3, 6, 0, 6], [1, 2, 7, 9, 6, 0]]


In [ ]:
### Alg_Prepare_Node ###
import copy

class Node():
    def __init__(self,matrix:list,distance,week,tracking_pos) -> None:
        self.matrix = matrix
        self.distance = distance
        self.week = week
        self.tracking_pos = tracking_pos                 

def get_sub_node(distance_matrix,node:Node):
    """ returns the list contains all tuples: (matrix,pos,distance) """
    ans_matrix = []
    final_ans = []
    queue = [(node,0)]
    current_week = node.week + 1
    N = len(distance_matrix)
    assert N%2 == 0, "The number of teams must be even"
    while queue:
        temp,match = queue.pop()
        matrix = temp.matrix
        pos = temp.tracking_pos
        distance = temp.distance
        if match == N//2:
            if temp.matrix not in ans_matrix:
                ans_matrix.append(temp.matrix)
                final_ans.append((temp.matrix,temp.tracking_pos,temp.distance))
        else:
            for i in range(N):
                if current_week not in matrix[i] and current_week not in [matrix[j][i] for\
                    j in range(N)]:
                    for j in range(N):
                        if i!=j and current_week not in matrix[j] and current_week not \
                            in [matrix[i][j] for i in range(N)]:
                            if matrix[i][j] == None:
                                sub_matrix = copy.deepcopy(matrix)
                                sub_matrix[i][j] = current_week
                                sub_pos = copy.deepcopy(pos)
                                sub_pos[i],sub_pos[j] = j,j
                                sub_distance = distance + distance_matrix[pos[i]][j] + distance_matrix[pos[j]][j]
                                sub_Node = Node(sub_matrix,sub_distance,current_week,sub_pos)
                                queue.append((sub_Node,match+1))
    return final_ans



Alg1_BackTracking

In [375]:
import time
before = time.time()

# from Alg_prepare_Node import Node, get_sub_node

### BackTracking ###
def backtrack_solve(input:list):
    N = len(input) # N
    queue = []     # initial queue empty
    ## tracking the best distance in the tree
    best_distance = float('inf')
    best_config = None
    #initial the root of the tree
    root_matrix = [[None for i in range(N)] for i in range(N)]
    for i in range(N):
        root_matrix[i][i] = 0     # Đường chéo chính  = 0
    root_distance = 0     # initial root_distance
    week = 0      # initial week
    root_tracking_pos = [i for i in range(N)]
    root = Node(root_matrix,root_distance,week,root_tracking_pos)
    queue.append(root)

    while queue:   # matrix, distance
        temp = queue.pop()     #stack: a, b, c -> pop(0): a, pop(): c
        assert isinstance(temp, Node)  # Not assertIsInstance in unittest - khẳng định là trường hợp
        current_week = temp.week
        if current_week == 2*N - 2:
            if temp.distance < best_distance:
                best_config = temp.matrix
                best_distance = temp.distance
                print(f"Found better configuration, the new best distance is : {best_distance}")
                print(f"when: matrix is {best_config}")
                # print(f"number week {temp.week}")
                # print(f"The Tracking_pos {temp.tracking_pos}")
                print()

        else: # if current_week not 2N - 2
            next_week = current_week + 1    # +1 week
            list_of_nodes = get_sub_node(input,temp)
            for item in list_of_nodes:
                sub_matrix,sub_pos,sub_distance = item
                if sub_distance < best_distance:
                    sub_Node = Node(sub_matrix,sub_distance,next_week,sub_pos)
                    queue.append(sub_Node)

print('When use Backtracking Alg:')
# distance_matrix = [[0, 1, 2, 3], [1, 0, 4, 5], [2, 4, 0, 6], [3, 5, 6, 0]]
backtrack_solve(input=distance_matrix)

runtime = time.time() - before
print(f'Runtime: {runtime}')

KeyboardInterrupt: 

Alg2_Greedy

In [ ]:
import time
before = time.time()

import copy
# from Alg_Prepare_Node import Node, get_sub_node

### Greedy Search ###
def greedy_solve(input:list):
    N = len(input)
    queue = []
    ##initialize the root of the tree.
    root_matrix = [[None for i in range(N)] for i in range(N)]
    for i in range(N):
        root_matrix[i][i] = 0
    root_distance = 0
    week = 0
    root_tracking_pos = [i for i in range(N)]
    root = Node(root_matrix,root_distance,week,root_tracking_pos)
    queue.append(root)
    while queue:
        temp = queue.pop()
        assert isinstance(temp, Node)

        current_week = temp.week
        if current_week == 2*N - 2:
            return (temp.matrix,temp.distance)
        else:
            next_week = current_week + 1
            list_of_nodes = get_sub_node(input,temp)
            #node: sub_matrix,sub_pos,sub_distance
            new_list = sorted(list_of_nodes,key=lambda x:x[2])
            sub_matrix,sub_pos,sub_distance = list_of_nodes[0]
            new_node = Node(sub_matrix,sub_distance,next_week,sub_pos)
            queue.append(new_node)
# distance_matrix = [[0, 1, 2, 3], [1, 0, 4, 5], [2, 4, 0, 6], [3, 5, 6, 0]]
mat_greedy,cost_greedy = greedy_solve(distance_matrix)
print(f'The reasonable cost when using Greedy Search is {cost_greedy}, when: matrix is {mat_greedy}')

runtime = time.time() - before
print(f'Runtime: {runtime}')

Alg3_Beam_Search

In [ ]:
import time
before = time.time()

import copy
from Alg_Prepare_Node import Node, get_sub_node

def find_min(matrix:list):
    ans = float('inf')
    for row in matrix:
        for entry in row:
            if entry != 0 and entry < ans:
                ans = entry
    return ans

### BeamSearch algorithm ###
def beam_search_solve(input,k):
    N = len(input)
    queue = []
    ## tracking the best value in the tree
    c_min = find_min(input)

    ##
    ##initialize the root of the tree.
    root_matrix = [[None for i in range(N)] for i in range(N)]
    for i in range(N):
        root_matrix[i][i] = 0
    root_distance = 0
    week = 0
    root_tracking_pos = [i for i in range(N)]
    root = Node(root_matrix,root_distance,week,root_tracking_pos)
    queue.append(root)
    best = None
    
    while True:
        nodes = copy.deepcopy(queue) 
        queue = []
        if nodes[0].week == 2*N - 2:
            for node in nodes:
                if best == None:
                    best = node.matrix,node.distance
                else:
                    if node.distance < best[1]:
                        best = node.matrix,node.distance
            break
        else:
            next_week = nodes[0].week + 1
            for node in nodes:
                lis = get_sub_node(input,node)
                sub_nodes = [Node(sub_matrix,sub_distance,next_week,sub_pos) for \
                    (sub_matrix,sub_pos,sub_distance) in lis]
                queue += sub_nodes
            queue = sorted(queue,key=lambda x: x.distance)[:k]

    return best
distance_matrix = [[0, 1, 2, 3], [1, 0, 4, 5], [2, 4, 0, 6], [3, 5, 6, 0]]
config_beam_search,val_beam_search = beam_search_solve(distance_matrix,10) 
print(f'The reasonable cost when using Beam Search is {val_beam_search}, when: matrix is {config_beam_search}')

runtime = time.time() - before
print(f'Runtime: {runtime}')

: 

Alg4_Local_Search

In [ ]:
import time
before = time.time()

### Local Search ###
import traceback
import random
import numpy 
import copy

def generate(N):

    mat_0 = list()
    for i in range(N):
        row = [None]*N
        row[i] = 0
        mat_0.append(row)

    all_coors_0 = list((a,b) for a in range(0,N) for b in range(0,N))
    for i in range(N):
        all_coors_0.remove((i,i))

    n_choices_0 = dict()
    choices_0 = dict()
    for i in range(1,N+1):
        n_choices_0[i] = N-1
        choices_0[i] = [a for a in range(1,N+1) if a != i]
    
    univ_tab = dict()

    univ_tab[0] = [copy.deepcopy(n_choices_0), copy.deepcopy(choices_0), copy.deepcopy(all_coors_0), copy.deepcopy(mat_0)]

    global result
    result = []

    def construct_mat(n):
        global result
        if n == 2*N - 2:
            result = univ_tab[n][3]
        else:
            nextweek = construct_week(n+1)
            if nextweek == False:
                construct_mat(n)
            elif nextweek == 'Reset':
                construct_mat(n-1)
            else:
                univ_tab[n+1] = copy.deepcopy(nextweek)
                construct_mat(n+1)

    # functions for contructing week 

    def create_priority_queue(n_choices_f):
        L = list()
        for a,b in n_choices_f.items():
            L.append((b,a))

        L.sort()

        Q = list()
        for item in L:
            Q.append(item[1])
        return Q

    def check_choices(choices_copy):
        L = list()
        for key in choices_copy.keys():
            if len(choices_copy[key]) == 0:
                L.append(key)
        for key in L:
            choices_copy.pop(key)

    def create_n_choices(choices):
        n_choices_f = dict()
        for key in choices.keys():
            n_choices_f[key] = len(choices[key])
        
        return n_choices_f
    def choose_match(a: int, n_choices: dict, choices: dict, choices_copy: dict, all_coors: list, rest: list, queue: list, mat: list, week: int, p_queue: list, b = None):
        
        if b == None:
            b = random.choice(choices_copy[a])

        rest.remove(a)
        rest.remove(b)
        queue.remove(a)
        queue.remove(b)
        p_queue = copy.deepcopy(queue)

        for key in choices_copy.keys():
            if a in choices_copy[key]:
                choices_copy[key].remove(a)
            if b in choices_copy[key]:
                choices_copy[key].remove(b)
        choices_copy.pop(a)
        choices_copy.pop(b)

        RC = []
        if (a-1,b-1) in all_coors:
            RC.append((a,b))
        if (b-1,a-1) in all_coors:
            RC.append((b,a))
        
        (c,d) = random.choice(RC)

        all_coors.remove((c-1,d-1))

        if len(RC) == 1:
            n_choices[a] -= 1
            n_choices[b] -= 1
            choices[a].remove(b)
            choices[b].remove(a)

        mat[c-1][d-1] = week
    # constructing week

    def construct_week(m):

        univ = copy.deepcopy(univ_tab[m-1])
        # n_choices = univ[0]
        # choices = univ[1]
        # all_coors = univ[2]
        # mat = univ[3]
        
        choices_copy = copy.deepcopy(univ[1])

        queue = create_priority_queue(univ[0])

        p_queue = queue[:]
        rest = [i for i in range(1,N+1)]

        while len(p_queue) >=1:

            if len(rest) <= 2:
                if len(rest) == 1:
                    return False
                else:
                    if rest[1] not in univ[1][rest[0]]:
                        return False
                    else:
                        check_choices(choices_copy)
                        choose_match(rest[1], univ[0], univ[1], choices_copy, univ[2], rest, queue, univ[3], m, p_queue, rest[0])
                        return univ
            else:
                check_choices(choices_copy)
                queue = create_priority_queue(create_n_choices(choices_copy))
                p_queue = queue[:]

                i = p_queue[0]

                # print('Week', end ='')
                # print(m)

                choose_match(i, univ[0], univ[1], choices_copy, univ[2], rest, queue, univ[3], m, p_queue)

                if len(choices_copy.keys()) % 2 != 0:
                    return 'Reset'
    try:
        construct_mat(0)
    except Exception:

        construct_mat(0)

    return result


def get_neighbors(initialize): #initialize: mat,cost
    '''return list of (mat,cost)'''
    mat,cost = initialize
    M = copy.deepcopy(mat)
    N = len(M)
    
    all_neighbors = list()

    def team_cost(team,matrix,d_matrix):
        away = list(zip(list(matrix[team][i] for i in range(0,N)),range(1,N+1)))
        home = list(zip(list(matrix[i][team] for i in range(0,N)),list(team + 1 for j in range(0,N))))
        home.sort()
        home.pop(0)
        team_schedule = away + home
        team_schedule.sort()
        C = 0
        for i in range(len(team_schedule)-1):
            C += d_matrix[team_schedule[i][1] - 1][team_schedule[i+1][1] - 1]
        return C
    def match_swap(coor):
        Y = copy.deepcopy(mat)
        temp = Y[coor[0]][coor[1]]
        Y[coor[0]][coor[1]] = Y[coor[1]][coor[0]]
        Y[coor[1]][coor[0]] = temp

        A = team_cost(coor[0],mat,distance_matrix) + team_cost(coor[1],mat,distance_matrix) 
        B = team_cost(coor[0],Y,distance_matrix) + team_cost(coor[1],Y,distance_matrix)
        cost_Y = cost - A + B
        return (Y, cost_Y)

    def team_swap(teams):
        a,b = teams
        a -= 1
        b -= 1
        Y = copy.deepcopy(mat)
        Y_1 = numpy.array(copy.deepcopy(Y))
        
        def replace_in_mat(x,y,S):
            temp = S[x].copy()
            S[x] = S[y].copy()
            S[y] = temp.copy()

            S[x][y] = S[x][x]
            S[x][x] = 0

            S[y][x] = S[y][y]
            S[y][y] = 0

        replace_in_mat(a,b,Y_1)
        Y_1 = Y_1.T.copy()
        
        replace_in_mat(a,b,Y_1)
        Y_1 = Y_1.T.copy()

        Y = copy.deepcopy(Y_1.tolist())
        return (Y, compute_cost(distance_matrix, Y))
    
    iterate = list((j,i) for i in range(0,N) for j in range(0,i))

    for coor in iterate:
        mat_1, cost_1 = copy.deepcopy(match_swap(coor))
        all_neighbors.append((mat_1, cost_1))


    iterate = list((i,j) for i in range(1,N + 1) for j in range(i, N + 1))

    for coor in iterate:
        mat_1, cost_1 = copy.deepcopy(team_swap(coor))
        all_neighbors.append((mat_1, cost_1))
    
    return all_neighbors

def compute_cost(distance_matrix,matrix):
    '''compute cost of a solution'''
    num_of_teams = len(matrix)
    cost = 0
    for team in range(num_of_teams):
        current_pos = team
        team_dis = 0
        current_week = 0
        while current_week < 2*num_of_teams - 2:
            current_week = current_week + 1
            flag = True
            for index in range(num_of_teams):
                if matrix[team][index] == current_week:
                    old_pos = current_pos
                    current_pos = index
                    team_dis += distance_matrix[old_pos][current_pos]
                    flag = False
                    break
            if flag: # da san nha
                old_pos = current_pos
                current_pos = team
                team_dis += distance_matrix[old_pos][current_pos]

        cost += team_dis
        
    return cost       
def local_solve(input):
    # random inial
    #input: distance matrix
    #return the schedule: matrix , cost
    N = len(input)
    random_matrix = generate(N)
    cost = compute_cost(input,random_matrix)
    initialize = random_matrix,cost
    flag = False
    while flag:
        neighbors = get_neighbors(initialize)
        for neighbor in neighbors:
            #neighbor: matrix:schedule,cost
            schedule,cost = neighbor
            flag = False
            if cost < initialize[1]:
                initialize = neighbor
                flag = True
    return initialize[0],initialize[1]

# distance_matrix = [[0, 1, 2, 3], [1, 0, 4, 5], [2, 4, 0, 6], [3, 5, 6, 0]]
config_local_search, val_local_search = local_solve(distance_matrix)
print(f'The reasonable cost when using Local Search is {val_local_search}, when: matrix is {config_local_search}')

runtime = time.time() - before
print(f'Runtime: {runtime}')

Meta heuristic algorithm


In [ ]:
# Alg5_Iterative_Local_Search

# Meta heuristic algorithm 
# -> Phần 1: Iterative Local Search
# -> Phần 2: Gibb Sampling
import time
before = time.time()

from Alg4_Local_Search import local_solve

# iterative local search #
def iterative_local_solve(input,K):
    best_config = None
    best_cost = float('inf')
    for i in range(K):
        matrix,cost = local_solve(input)
        if cost < best_cost:
            best_cost = cost
            best_config = matrix

    return best_config,best_cost


# distance_matrix = [[0, 1, 2, 3], [1, 0, 4, 5], [2, 4, 0, 6], [3, 5, 6, 0]]
mat_iter_local_search,cost_iter_local_search = iterative_local_solve(distance_matrix,K=100)
print(f'The reasonable cost when using Iterative Local Search is {cost_iter_local_search}, when: matrix is {mat_iter_local_search}')

runtime = time.time() - before
print(f'Runtime: {runtime}')

In [ ]:
# Alg6_Gibb Sampling

# Meta heuristic algorithm 
# -> Phần 1: Iterative Local Search
# -> Phần 2: Gibb Sampling

import time
before = time.time()

import random
from Alg4_Local_Search import generate, compute_cost, get_neighbors

# gibb sampling: but this method is not efficent yet
def gibb_sampling(input,K):
    best_config = None
    best_cost = float('inf')
    #initialize start position
    mat = generate(len(input))
    current_postion = (mat,compute_cost(distance_matrix,mat))
    
    for i in range(K):
        #choosing a random number:
        num = random.random()

        list_of_neighs = get_neighbors(current_postion) #mat,cost
        sum_of_cost = sum(x[1] for x in list_of_neighs)
        accumulate = 0
        for i in range(len(list_of_neighs)):
            mat,cost = list_of_neighs[i]
            accumulate += cost
            port = accumulate / sum_of_cost
            if port > num:
                current_postion = list_of_neighs[i-1]
                break
    return current_postion

# distance_matrix = [[0, 1, 2, 3], [1, 0, 4, 5], [2, 4, 0, 6], [3, 5, 6, 0]]    
mat_gibb_sampling,cost_gibb_sampling = gibb_sampling(distance_matrix,1000)
print(f'The reasonable cost when using Gibb_sampling is {cost_gibb_sampling}, when: matrix is {mat_gibb_sampling}')

runtime = time.time() - before
print(f'Runtime: {runtime}')

Alg7_Ortool

In [ ]:
import time
before = time.time()

from ortools.sat.python import cp_model
import copy
import time

class VarArrayAndObjectiveSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions (objective, variable values, time)."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__start_time = time.time()

    def on_solution_callback(self):
        """Called on each new solution."""
        current_time = time.time()
        obj = self.ObjectiveValue()
        print('Solution %i, time = %0.2f s, objective = %i' %
              (self.__solution_count, current_time - self.__start_time, obj))
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(self.Value(self.__variables[i][j]), end = ' ')
            print()
        print()
        self.__solution_count += 1

    def solution_count(self):
        """Returns the number of solutions found."""
        return self.__solution_count
def OR_Tools_solve(input):
    
    distance_matrix = input
    N = len(input)

    model = cp_model.CpModel()

    x = []
    for i in range(N):
        row = [None]*N
        row[i] = 0
        x.append(row)

    for i in range(N):
        for j in range(N):
            if i == j:
                x[i][i] = model.NewIntVar(0,0,'x[%i][%i]' % (i,i))
            else:
                x[i][j] = model.NewIntVar(0,2*N - 2, 'x[%i][%i]' %(i,j))

    for i in range(N):
        c = list(x[j][i] for j in range(N))
        c.pop(i)
        cnr = x[i] + c
        model.AddAllDifferent(cnr)
    
    y = []
    for i in range(N):
        y_row = []
        for j in range(2*N - 1):
            row = [None]*(2*N - 1)
            row[i] = 0
            y_row.append(row)
        y.append(y_row)
    for i in range(N):
        for j in range(2*N - 1):
            for k in range(2*N - 1):
                y[i][j][k] = model.NewBoolVar('y[%i][%i][%i]' % (i,j,k))


    obj_func = 0

    for team in range(N):

        indexes = list((team,j) for j in range(0,N) if team!= j) + list((j,team) for j in range(0,N) if team!= j)
        index_0 = (team,team)

        for i in range(len(indexes)):
            model.Add(x[indexes[i][0]][indexes[i][1]] == x[index_0[0]][index_0[1]] + 1).OnlyEnforceIf(y[team][0][i + 1])
            model.Add(x[indexes[i][0]][indexes[i][1]] != x[index_0[0]][index_0[1]] + 1).OnlyEnforceIf(y[team][0][i + 1].Not())
        for i in range(len(indexes)):
            indexes_copy = copy.deepcopy(indexes)
            for j in range(len(indexes_copy)):
                if i != j:
                    model.Add(x[indexes[i][0]][indexes[i][1]] == x[indexes_copy[j][0]][indexes_copy[j][1]] - 1).OnlyEnforceIf(y[team][i + 1][j + 1])
                    model.Add(x[indexes[i][0]][indexes[i][1]] != x[indexes_copy[j][0]][indexes_copy[j][1]] - 1).OnlyEnforceIf(y[team][i + 1][j + 1].Not())
        
        others = list(i for i in range(N) if i!= team)
        for i in range(2*N - 1):
            for j in range(2*N - 1):
                if i == 0:
                    if j == 0:
                        obj_func += 0
                    elif j <= N-1:
                        obj_func += distance_matrix[team][others[j - 1]] * y[team][0][j]
                    else:
                        obj_func += distance_matrix[team][team] * y[team][0][j]
                elif i <= N-1:
                    if j == 0:
                        obj_func += 0
                    elif j <= N-1:
                        obj_func += distance_matrix[others[i - 1]][others[j - 1]] * y[team][i][j]
                    else:
                        obj_func += distance_matrix[others[i - 1]][team] * y[team][i][j]
                else:
                    if j == 0:
                        obj_func += 0
                    elif j <= N-1:
                        obj_func += distance_matrix[team][others[j - 1]] * y[team][i][j]
                    else:
                        obj_func += distance_matrix[team][team] * y[team][i][j]

    model.Minimize(obj_func)
    
    solver = cp_model.CpSolver()
    solution_printer = VarArrayAndObjectiveSolutionPrinter(x)
    status = solver.Solve(model, solution_printer)
    
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Minimum of objective function: {solver.ObjectiveValue()}\n')
        # print(solver.Value())
        # TypeError: CpSolver.Value() missing 1 required positional argument: 'expression'
        for i in range(N):
            for j in range(N):
                print(solver.Value(x[i][j]), end = ' ')
            print()
    

    else:
        print('No solution found.')
    
    print('\nStatistic')
    print(f'  status : {solver.StatusName(status)}')
    print(f'  conflicts: {solver.NumConflicts()}')
    print(f'  branches : {solver.NumBranches()}')
    print(f'  wall time: {solver.WallTime()} s')
    print(f'  Reasonable cost using OR-Tools: {solver.ObjectiveValue()}')


# distance_matrix = [[0, 1, 2, 3], [1, 0, 4, 5], [2, 4, 0, 6], [3, 5, 6, 0]]
print('When using Or - Tool:')
OR_Tools_solve(distance_matrix)


runtime = time.time() - before
print(f'Runtime: {runtime}')

EXPERIMENTS

In [ ]:
### Compare optimization ###
print(f'generate distance_matrix is {distance_matrix}')
print('When using Backtracking:')
backtrack_solve(distance_matrix)
print(f'The reasonable cost when using Greedy Search is {cost_greedy}, when: matrix is {mat_greedy}')
print(f'The reasonable cost when using Beam Search is {val_beam_search}, when: matrix is {config_beam_search}')
print(f'The reasonable cost when using Local Search is {val_local_search}, when: matrix is {config_local_search}')
print(f'The reasonable cost when using Iterative Local Search is {cost_iter_local_search}, when: matrix is {mat_iter_local_search}')
print(f'The reasonable cost when using Gibb_sampling is {cost_gibb_sampling}, when: matrix is {mat_gibb_sampling}')
print('When using Or - Tool:')
OR_Tools_solve(distance_matrix)

After many tests with n = 4: 
1. generate distance_matrix is [[0, 5, 1, 8], [5, 0, 2, 6], [1, 2, 0, 1], [8, 6, 1, 0]], generate distance_matrix is [[0, 3, 9, 3], [3, 0, 5, 8], [9, 5, 0, 1], [3, 8, 1, 0]]
The best algorithms is: Backtracking and Or-tool
2. generate distance_matrix is [[0, 1, 9, 9], [1, 0, 5, 6], [9, 5, 0, 1], [9, 6, 1, 0]], generate distance_matrix is [[0, 6, 9, 9], [6, 0, 2, 2], [9, 2, 0, 3], [9, 2, 3, 0]]. 
The best algorithms is: Backtracking, Greedy Search and Or-tool 
3. generate distance_matrix is [[0, 9, 5, 4], [9, 0, 2, 3], [5, 2, 0, 3], [4, 3, 3, 0]], generate distance_matrix is [[0, 1, 2, 5], [1, 0, 4, 6], [2, 4, 0, 2], [5, 6, 2, 0]]
The best algorithms is: Backtracking, Beam Search and Or-tool 

-> with n = 4 -> The best algorithms is: Backtracking and Or-tool with RunTime 1-2 minutes 

After many tests with n = 6: 
1. generate distance_matrix is [[0, 9, 4, 7, 9, 7], [9, 0, 2, 1, 4, 8], [4, 2, 0, 5, 7, 2], [7, 1, 5, 0, 9, 3], [9, 4, 7, 9, 0, 6], [7, 8, 2, 3, 6, 0]]
The best algorithms is: 
- Backtracking: Found better configuration, the new best distance is : 172 and Still running and taking too much time
- The reasonable cost when using Greedy Search is 240, 
Runtime: 0.29929494857788086
- The reasonable cost when using Beam Search is 164, 
Runtime: 2.494082450866699
- The reasonable cost when using Local Search is 221, 
Runtime: 2.5168230533599854
- The reasonable cost when using Gibb_sampling is 268, 
Runtime: 10.593667984008789
RecursionError: maximum recursion depth exceeded while calling a Python object
- OR-Tools: Solution 25, time = 6.98 s, objective = 160 
Solution 28, time = 59.69 s, objective = 156
and Still running and taking too much time


2. generate distance_matrix is [[0, 5, 9, 9, 4, 1], [5, 0, 8, 4, 1, 2], [9, 8, 0, 7, 3, 7], [9, 4, 7, 0, 6, 9], [4, 1, 3, 6, 0, 6], [1, 2, 7, 9, 6, 0]]
The best algorithms is: 
- Found better configuration, the new best distance is : 198 and Still running and taking too much time
- The reasonable cost when using Greedy Search is 244, 
Runtime: 0.4422941207885742
- The reasonable cost when using Beam Search is 150, 
Runtime: 2.5910069942474365
- Solution 23, time = 5.80 s, objective = 142

3. generate distance_matrix is [[0, 4, 6, 5, 3, 3], [4, 0, 8, 9, 7, 1], [6, 8, 0, 9, 4, 3], [5, 9, 9, 0, 9, 4], [3, 7, 4, 9, 0, 6], [3, 1, 3, 4, 6, 0]]
The best algorithms is:
- Found better configuration, the new best distance is : 187 and Still running and taking too much time
- The reasonable cost when using Greedy Search is 234, 
Runtime: 0.3998544216156006
- The reasonable cost when using Beam Search is 173, 
Runtime: 2.559349536895752
- Solution 18, time = 224.72 s, objective = 150 and Still running and taking too much time

=>>> The best algorithms is Beam Search and OR-tools 
()

After many tests with n = 10

- 
- Solution 47, time = 154.78 s, objective = 467
Solution 48, time = 171.18 s, objective = 462
=>>> The best algorithms is Beam Search and OR-tools